In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem

from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, AllChem, rdFingerprintGenerator

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, \
    classification_report, roc_curve, roc_auc_score, RocCurveDisplay, average_precision_score, make_scorer
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import GridSearchCV


In [ ]:
dataset_chemical_physical = pd.read_csv('datasets/pubchem_chemical_and_physical_properties.csv', low_memory=False)

In [ ]:
dataset_chemical_physical

In [ ]:
dataset_chemical_physical_for_work = dataset_chemical_physical[['cmpdname', 'smiles', 'mw', 'xlogp', 'hbonddonor', 'hbondacc', 'polararea', 'rotbonds']]

In [ ]:
violations_lipinski = []

for row in range(0, len(dataset_chemical_physical_for_work)):

    nviolations = 0

    if float(dataset_chemical_physical_for_work.iloc[row, 2]) > 500:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 3]) > 5:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 3]) < 0:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 4]) > 5:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 5]) > 10:
        nviolations += 1

    violations_lipinski.append(nviolations)

In [ ]:
violations_bro = []

for row in range(0, len(dataset_chemical_physical_for_work)):

    nviolations = 0

    if float(dataset_chemical_physical_for_work.iloc[row, 2]) > 1000:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 3]) > 10:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 3]) < -2:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 4]) > 6:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 5]) > 15:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 6]) > 250:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 7]) > 20:
        nviolations += 1

    violations_bro.append(nviolations)

In [ ]:
violations_muegge = []

for row in range(0, len(dataset_chemical_physical_for_work)):

    nviolations = 0

    if float(dataset_chemical_physical_for_work.iloc[row, 2]) > 600:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 2]) < 200:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 3]) > 5:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 3]) < -2:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 4]) > 5:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 5]) > 10:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 6]) > 150:
        nviolations += 1
    if float(dataset_chemical_physical_for_work.iloc[row, 7]) > 15:
        nviolations += 1

    count_carbon = 0
    count_heteroatoms = 0
    count_rings = 0

    for symbol in dataset_chemical_physical_for_work.iloc[row, 1]:

        heteroatoms_check = ['C', 'c', 'H', 'h', '[', ']', '(', ')', '+', '-', '=', '#']

        if symbol == 'C':
            count_carbon += 1
        elif symbol not in heteroatoms_check:
            count_heteroatoms += 1
        elif symbol.isdigit():
            count_rings = int(symbol)

        if symbol == 'l':
            count_carbon -= 1
        # elif symbol == 'r':
        #     count_carbon -= 1

    if count_carbon < 5:
        nviolations += 1
    if count_heteroatoms < 2:
        nviolations += 1
    if count_rings > 7:
        nviolations += 1


    violations_muegge.append(nviolations)

In [ ]:
violations_lipinski_dict = {'nviolations_lipinski_rule': violations_lipinski}
violations_bro_dict = {'nviolations_bro_rule': violations_bro}
violations_muegge_dict = {'nviolations_muegge_rule': violations_muegge}

In [ ]:
dataset_with_nviolations_con = pd.concat([dataset_chemical_physical_for_work, pd.DataFrame(violations_lipinski_dict), pd.DataFrame(violations_bro_dict), pd.DataFrame(violations_muegge_dict)], axis=1)

In [ ]:
dataset_with_nviolations = dataset_with_nviolations_con.dropna()

In [ ]:
molecule_ml1 = Chem.MolFromSmiles('N[C@@H](CCCNC(=N)N)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)O')
molecule_ml2 = Chem.MolFromSmiles('NC(CCNC(=N)N)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)O')
molecule_ml3 = Chem.MolFromSmiles('NC(CNC(=N)N)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)O')
molecule_ml4 = Chem.MolFromSmiles('N[C@@H](CCONC(=N)N)C(=O)OCC(=O)N[C@@H](CC(=O)O)C(=O)O')
molecule_ml5 = Chem.MolFromSmiles('NCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)[C@@H](N)CCCNC(=N)N')
molecule_ml6 = Chem.MolFromSmiles('NCCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)[C@@H](N)CCCNC(=N)N')
molecule_ml7 = Chem.MolFromSmiles('NCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CCNC(=N)N')
molecule_ml8 = Chem.MolFromSmiles('NCCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CCNC(=N)N')
molecule_ml9 = Chem.MolFromSmiles('NCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CNC(=N)N')
molecule_ml10 = Chem.MolFromSmiles('NCCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CNC(=N)N')
molecule_ml11 = Chem.MolFromSmiles('NCCNC(=O)[C@H](CC(=O)O)NC(=O)COC(=O)[C@@H](N)CCONC(=N)N')
molecule_ml12 = Chem.MolFromSmiles('NCCCNC(=O)[C@H](CC(=O)O)NC(=O)COC(=O)[C@@H](N)CCONC(=N)N')
molecule_ml13 = Chem.MolFromSmiles('CC[C@H](C)[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(=O)O')
molecule_ml14 = Chem.MolFromSmiles('CC[C@H](C)[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@H](Cc2ccc(F)cc2)NC(=O)[C@H](C)N)C(=O)O')
molecule_ml15 = Chem.MolFromSmiles('C[C@H](N)C(=O)N[C@@H](Cc1ccc(F)cc1)C(=O)N2CCC[C@H]2C(=O)N[C@H](Cc3ccc(F)cc3)C(=O)O')
molecule_ml16 = Chem.MolFromSmiles('CC(C)[C@H](NC(=O)[C@H](C)N)C(=O)N1CCC[C@H]1C(=O)N[C@H](Cc2ccc(F)cc2)C(=O)O')
molecule_ml17 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](N)CCCNC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml18 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](N)CCCCN)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml19 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCCN)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml20 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml21 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](N)CCCNC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml22 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml23 = Chem.MolFromSmiles('CC[C@@H](C)[C@H](NC(=O)[C@H](Cc1ccc(F)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml24 = Chem.MolFromSmiles('CCC(C)[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)O')
molecule_ml25 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](NC(=O)[C@@H]3CCCN3C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(C)CC)C(=O)N[C@@H](CC(C)C)C(=O)O')
molecule_ml26 = Chem.MolFromSmiles('CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](NC(=O)[C@@H]3CCCN3C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(C)CC)C(=O)N[C@@H](CC(C)C)C(=O)O')

In [ ]:
descriptors_ml1 = Descriptors.CalcMolDescriptors(molecule_ml1)
descriptors_ml2 = Descriptors.CalcMolDescriptors(molecule_ml2)
descriptors_ml3 = Descriptors.CalcMolDescriptors(molecule_ml3)
descriptors_ml4 = Descriptors.CalcMolDescriptors(molecule_ml4)
descriptors_ml5 = Descriptors.CalcMolDescriptors(molecule_ml5)
descriptors_ml6 = Descriptors.CalcMolDescriptors(molecule_ml6)
descriptors_ml7 = Descriptors.CalcMolDescriptors(molecule_ml7)
descriptors_ml8 = Descriptors.CalcMolDescriptors(molecule_ml8)
descriptors_ml9 = Descriptors.CalcMolDescriptors(molecule_ml9)
descriptors_ml10 = Descriptors.CalcMolDescriptors(molecule_ml10)
descriptors_ml11 = Descriptors.CalcMolDescriptors(molecule_ml11)
descriptors_ml12 = Descriptors.CalcMolDescriptors(molecule_ml12)
descriptors_ml13 = Descriptors.CalcMolDescriptors(molecule_ml13)
descriptors_ml14 = Descriptors.CalcMolDescriptors(molecule_ml14)
descriptors_ml15 = Descriptors.CalcMolDescriptors(molecule_ml15)
descriptors_ml16 = Descriptors.CalcMolDescriptors(molecule_ml16)
descriptors_ml17 = Descriptors.CalcMolDescriptors(molecule_ml17)
descriptors_ml18 = Descriptors.CalcMolDescriptors(molecule_ml18)
descriptors_ml19 = Descriptors.CalcMolDescriptors(molecule_ml19)
descriptors_ml20 = Descriptors.CalcMolDescriptors(molecule_ml20)
descriptors_ml21 = Descriptors.CalcMolDescriptors(molecule_ml21)
descriptors_ml22 = Descriptors.CalcMolDescriptors(molecule_ml22)
descriptors_ml23 = Descriptors.CalcMolDescriptors(molecule_ml23)
descriptors_ml24 = Descriptors.CalcMolDescriptors(molecule_ml24)
descriptors_ml25 = Descriptors.CalcMolDescriptors(molecule_ml25)
descriptors_ml26 = Descriptors.CalcMolDescriptors(molecule_ml26)

In [ ]:
molecular_descriptors_test_peptides = {
    'MolWt': [],
    'NHOHCount': [],
    'NOCount': [],
    'NumHAcceptors': [],
    'NumHDonors': [],
    'NumRotatableBonds': [],
    'MolLogP': [],
    'TPSA': []
}

def molecular_descriptors_calculation_peptides(descriptors_dict):
    for descriptor, value in descriptors_dict.items():
        if descriptor == 'MolWt':
            molecular_descriptors_test_peptides['MolWt'].append(value)
        elif descriptor == 'NHOHCount':
            molecular_descriptors_test_peptides['NHOHCount'].append(value)
        elif descriptor == 'NOCount':
            molecular_descriptors_test_peptides['NOCount'].append(value)
        elif descriptor == 'NumHAcceptors':
            molecular_descriptors_test_peptides['NumHAcceptors'].append(value)
        elif descriptor == 'NumHDonors':
            molecular_descriptors_test_peptides['NumHDonors'].append(value)
        elif descriptor == 'NumRotatableBonds':
            molecular_descriptors_test_peptides['NumRotatableBonds'].append(value)
        elif descriptor == 'MolLogP':
            molecular_descriptors_test_peptides['MolLogP'].append(value)
        elif descriptor == 'TPSA':
            molecular_descriptors_test_peptides['TPSA'].append(value)

In [ ]:
molecular_descriptors_calculation_peptides(descriptors_ml1)
molecular_descriptors_calculation_peptides(descriptors_ml2)
molecular_descriptors_calculation_peptides(descriptors_ml3)
molecular_descriptors_calculation_peptides(descriptors_ml4)
molecular_descriptors_calculation_peptides(descriptors_ml5)
molecular_descriptors_calculation_peptides(descriptors_ml6)
molecular_descriptors_calculation_peptides(descriptors_ml7)
molecular_descriptors_calculation_peptides(descriptors_ml8)
molecular_descriptors_calculation_peptides(descriptors_ml9)
molecular_descriptors_calculation_peptides(descriptors_ml10)
molecular_descriptors_calculation_peptides(descriptors_ml11)
molecular_descriptors_calculation_peptides(descriptors_ml12)
molecular_descriptors_calculation_peptides(descriptors_ml13)
molecular_descriptors_calculation_peptides(descriptors_ml14)
molecular_descriptors_calculation_peptides(descriptors_ml15)
molecular_descriptors_calculation_peptides(descriptors_ml16)
molecular_descriptors_calculation_peptides(descriptors_ml17)
molecular_descriptors_calculation_peptides(descriptors_ml18)
molecular_descriptors_calculation_peptides(descriptors_ml19)
molecular_descriptors_calculation_peptides(descriptors_ml20)
molecular_descriptors_calculation_peptides(descriptors_ml21)
molecular_descriptors_calculation_peptides(descriptors_ml22)
molecular_descriptors_calculation_peptides(descriptors_ml23)
molecular_descriptors_calculation_peptides(descriptors_ml24)
molecular_descriptors_calculation_peptides(descriptors_ml25)
molecular_descriptors_calculation_peptides(descriptors_ml26)

In [ ]:
attributes_lipinski_train, attributes_lipinski_test, target_lipinski_train, target_lipinski_test = train_test_split(dataset_with_nviolations[['mw', 'xlogp', 'hbonddonor', 'hbondacc']], dataset_with_nviolations[['nviolations_lipinski_rule']], test_size=0.2, random_state=42)

In [ ]:
attributes_bro_train, attributes_bro_test, target_bro_train, target_bro_test = train_test_split(dataset_with_nviolations[['mw', 'xlogp', 'hbonddonor', 'hbondacc', 'polararea', 'rotbonds']], dataset_with_nviolations[['nviolations_bro_rule']], test_size=0.2, random_state=42)

In [ ]:
attributes_muegge_train, attributes_muegge_test, target_muegge_train, target_muegge_test = train_test_split(dataset_with_nviolations[['mw', 'xlogp', 'hbonddonor', 'hbondacc', 'polararea', 'rotbonds']], dataset_with_nviolations[['nviolations_muegge_rule']], test_size=0.2, random_state=42)

In [ ]:
rf_lipinski = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_lipinski, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=1, verbose=2)

grid_search.fit(attributes_lipinski_train, target_lipinski_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

In [ ]:
rf_lipinski = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(precision_score, average='macro')

grid_search = GridSearchCV(estimator=rf_lipinski, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_lipinski_train, target_lipinski_train)

print("Best parameters:", grid_search.best_params_)
print("Best precision:", grid_search.best_score_)

In [ ]:
rf_lipinski = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(recall_score, average='macro')

grid_search = GridSearchCV(estimator=rf_lipinski, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_lipinski_train, target_lipinski_train)

print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)

In [ ]:
rf_lipinski = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(estimator=rf_lipinski, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_lipinski_train, target_lipinski_train)

print("Best parameters:", grid_search.best_params_)
print("Best F1:", grid_search.best_score_)

In [ ]:
rf_muegge = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_muegge, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=1, verbose=2)

grid_search.fit(attributes_muegge_train, target_muegge_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

In [ ]:
rf_muegge = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(precision_score, average='macro')

grid_search = GridSearchCV(estimator=rf_muegge, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_muegge_train, target_muegge_train)

print("Best parameters:", grid_search.best_params_)
print("Best precision:", grid_search.best_score_)

In [ ]:
rf_muegge = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(recall_score, average='macro')

grid_search = GridSearchCV(estimator=rf_muegge, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_muegge_train, target_muegge_train)

print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)

In [ ]:
rf_muegge = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(estimator=rf_muegge, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_muegge_train, target_muegge_train)

print("Best parameters:", grid_search.best_params_)
print("Best F1:", grid_search.best_score_)

In [ ]:
rf_bro = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_bro, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=1, verbose=2)

grid_search.fit(attributes_bro_train, target_bro_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

In [ ]:
rf_bro = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(precision_score, average='macro')

grid_search = GridSearchCV(estimator=rf_bro, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_bro_train, target_bro_train)

print("Best parameters:", grid_search.best_params_)
print("Best precision:", grid_search.best_score_)

In [ ]:
rf_bro = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(recall_score, average='macro')

grid_search = GridSearchCV(estimator=rf_bro, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_bro_train, target_bro_train)

print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)

In [ ]:
rf_bro = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(estimator=rf_bro, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2)

grid_search.fit(attributes_bro_train, target_bro_train)

print("Best parameters:", grid_search.best_params_)
print("Best F1:", grid_search.best_score_)

In [ ]:
model_lipinski_10 = RandomForestClassifier(n_estimators=10)

In [ ]:
model_lipinski_20 = RandomForestClassifier(n_estimators=20)

In [ ]:
model_lipinski_30 = RandomForestClassifier(n_estimators=30)

In [ ]:
model_bro_10 = RandomForestClassifier(n_estimators=10)

In [ ]:
model_bro_20 = RandomForestClassifier(n_estimators=20)

In [ ]:
model_bro_30 = RandomForestClassifier(n_estimators=30)

In [ ]:
model_muegge_10 = RandomForestClassifier(n_estimators=10)

In [ ]:
model_muegge_20 = RandomForestClassifier(n_estimators=20)

In [ ]:
model_muegge_30 = RandomForestClassifier(n_estimators=30)

In [ ]:
model_lipinski_10.fit(attributes_lipinski_train, target_lipinski_train)

In [ ]:
model_lipinski_20.fit(attributes_lipinski_train, target_lipinski_train)

In [ ]:
model_lipinski_30.fit(attributes_lipinski_train, target_lipinski_train)

In [ ]:
model_bro_10.fit(attributes_bro_train, target_bro_train)

In [ ]:
model_bro_20.fit(attributes_bro_train, target_bro_train)

In [ ]:
model_bro_30.fit(attributes_bro_train, target_bro_train)

In [ ]:
model_muegge_10.fit(attributes_muegge_train, target_muegge_train)

In [ ]:
model_muegge_20.fit(attributes_muegge_train, target_muegge_train)

In [ ]:
model_muegge_30.fit(attributes_muegge_train, target_muegge_train)

In [ ]:
test_prediction_lipinski_10 = model_lipinski_10.predict(attributes_lipinski_test)
test_prediction_lipinski_20 = model_lipinski_20.predict(attributes_lipinski_test)
test_prediction_lipinski_30 = model_lipinski_30.predict(attributes_lipinski_test)

In [ ]:
test_prediction_bro_10 = model_bro_10.predict(attributes_bro_test)
test_prediction_bro_20 = model_bro_20.predict(attributes_bro_test)
test_prediction_bro_30 = model_bro_30.predict(attributes_bro_test)

In [ ]:
test_predicton_muegge_10 = model_muegge_10.predict(attributes_muegge_test)
test_predicton_muegge_20 = model_muegge_20.predict(attributes_muegge_test)
test_predicton_muegge_30 = model_muegge_30.predict(attributes_muegge_test)

In [ ]:
accuracy_lipinski_model_10 = accuracy_score(target_lipinski_test, test_prediction_lipinski_10)
accuracy_lipinski_model_20 = accuracy_score(target_lipinski_test, test_prediction_lipinski_20)
accuracy_lipinski_model_30 = accuracy_score(target_lipinski_test, test_prediction_lipinski_30)

In [ ]:
accuracy_bro_model_10 = accuracy_score(target_bro_test, test_prediction_bro_10)
accuracy_bro_model_20 = accuracy_score(target_bro_test, test_prediction_bro_20)
accuracy_bro_model_30 = accuracy_score(target_bro_test, test_prediction_bro_30)

In [ ]:
accuracy_muegge_model_10 = accuracy_score(target_muegge_test, test_predicton_muegge_10)
accuracy_muegge_model_20 = accuracy_score(target_muegge_test, test_predicton_muegge_20)
accuracy_muegge_model_30 = accuracy_score(target_muegge_test, test_predicton_muegge_30)

In [ ]:
print("Accuracy of Lipinski model with 10 trees:", accuracy_lipinski_model_10)
print("Accuracy of Lipinski model with 20 trees:", accuracy_lipinski_model_20)
print("Accuracy of Lipinski model with 30 trees:", accuracy_lipinski_model_30)

In [ ]:
print("Accuracy of Bro model with 10 trees:", accuracy_bro_model_10)
print("Accuracy of Bro model with 20 trees:", accuracy_bro_model_20)
print("Accuracy of Bro model with 30 trees:", accuracy_bro_model_30)

In [ ]:
print("Accuracy of Muegge model with 10 trees:", accuracy_muegge_model_10)
print("Accuracy of Muegge model with 20 trees:", accuracy_muegge_model_20)
print("Accuracy of Muegge model with 30 trees:", accuracy_muegge_model_30)

In [ ]:
precision_lipinski_model_10 = precision_score(target_lipinski_test, test_prediction_lipinski_10, average='weighted')
precision_lipinski_model_20 = precision_score(target_lipinski_test, test_prediction_lipinski_20, average='weighted')
precision_lipinski_model_30 = precision_score(target_lipinski_test, test_prediction_lipinski_30, average='weighted')

In [ ]:
precision_bro_model_10 = precision_score(target_bro_test, test_prediction_bro_10, average='weighted')
precision_bro_model_20 = precision_score(target_bro_test, test_prediction_bro_20, average='weighted')
precision_bro_model_30 = precision_score(target_bro_test, test_prediction_bro_30, average='weighted')

In [ ]:
precision_muegge_model_10 = precision_score(target_muegge_test, test_predicton_muegge_10, average='weighted')
precision_muegge_model_20 = precision_score(target_muegge_test, test_predicton_muegge_20, average='weighted')
precision_muegge_model_30 = precision_score(target_muegge_test, test_predicton_muegge_30, average='weighted')

In [ ]:
print("Precision of Lipinski model with 10 trees:", precision_lipinski_model_10)
print("Precision of Lipinski model with 20 trees:", precision_lipinski_model_20)
print("Precision of Lipinski model with 30 trees:", precision_lipinski_model_30)

In [ ]:
print("Precision of Bro model with 10 trees:", precision_bro_model_10)
print("Precision of Bro model with 20 trees:", precision_bro_model_20)
print("Precision of Bro model with 30 trees:", precision_bro_model_30)

In [ ]:
print("Precision of Muegge model with 10 trees:", precision_muegge_model_10)
print("Precision of Muegge model with 20 trees:", precision_muegge_model_20)
print("Precision of Muegge model with 30 trees:", precision_muegge_model_30)

In [ ]:
recall_lipinski_10 = recall_score(target_lipinski_test, test_prediction_lipinski_10, average='weighted')
recall_lipinski_20 = recall_score(target_lipinski_test, test_prediction_lipinski_20, average='weighted')
recall_lipinski_30 = recall_score(target_lipinski_test, test_prediction_lipinski_30, average='weighted')

In [ ]:
recall_bro_10 = recall_score(target_bro_test, test_prediction_bro_10, average='weighted')
recall_bro_20 = recall_score(target_bro_test, test_prediction_bro_20, average='weighted')
recall_bro_30 = recall_score(target_bro_test, test_prediction_bro_30, average='weighted')

In [ ]:
recall_muegge_10 = recall_score(target_muegge_test, test_predicton_muegge_10, average='weighted')
recall_muegge_20 = recall_score(target_muegge_test, test_predicton_muegge_20, average='weighted')
recall_muegge_30 = recall_score(target_muegge_test, test_predicton_muegge_30, average='weighted')

In [ ]:
print("Recall of Lipinski model with 10 trees:", recall_lipinski_10)
print("Recall of Lipinski model with 20 trees:", recall_lipinski_20)
print("Recall of Lipinski model with 30 trees:", recall_lipinski_30)

In [ ]:
print("Recall of Bro model with 10 trees:", recall_bro_10)
print("Recall of Bro model with 20 trees:", recall_bro_20)
print("Recall of Bro model with 30 trees:", recall_bro_30)

In [ ]:
print("Recall of Muegge model with 10 trees:", recall_muegge_10)
print("Recall of Muegge model with 20 trees:", recall_muegge_20)
print("Recall of Muegge model with 30 trees:", recall_muegge_30)

In [ ]:
f1_score_lipinski_10 = f1_score(target_lipinski_test, test_prediction_lipinski_10, average='weighted')
f1_score_lipinski_20 = f1_score(target_lipinski_test, test_prediction_lipinski_20, average='weighted')
f1_score_lipinski_30 = f1_score(target_lipinski_test, test_prediction_lipinski_30, average='weighted')

In [ ]:
f1_score_bro_10 = f1_score(target_bro_test, test_prediction_bro_10, average='weighted')
f1_score_bro_20 = f1_score(target_bro_test, test_prediction_bro_20, average='weighted')
f1_score_bro_30 = f1_score(target_bro_test, test_prediction_bro_30, average='weighted')

In [ ]:
f1_score_muegge_10 = f1_score(target_muegge_test, test_predicton_muegge_10, average='weighted')
f1_score_muegge_20 = f1_score(target_muegge_test, test_predicton_muegge_20, average='weighted')
f1_score_muegge_30 = f1_score(target_muegge_test, test_predicton_muegge_30, average='weighted')

In [ ]:
print("F1 Score of Lipinski model with 10 trees:", f1_score_lipinski_10)
print("F1 Score of Lipinski model with 20 trees:", f1_score_lipinski_20)
print("F1 Score of Lipinski model with 30 trees:", f1_score_lipinski_30)

In [ ]:
print("F1 Score of Bro model with 10 trees:", f1_score_bro_10)
print("F1 Score of Bro model with 20 trees:", f1_score_bro_20)
print("F1 Score of Bro model with 30 trees:", f1_score_bro_30)

In [ ]:
print("F1 Score of Muegge model with 10 trees:", f1_score_muegge_10)
print("F1 Score of Muegge model with 20 trees:", f1_score_muegge_20)
print("F1 Score of Muegge model with 30 trees:", f1_score_muegge_30)

In [ ]:
plt.scatter(target_lipinski_test, test_prediction_lipinski_10, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (Lipinski 10 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_lipinski_test, test_prediction_lipinski_20, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (Lipinski 20 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_lipinski_test, test_prediction_lipinski_30, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (Lipinski 30 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_muegge_test, test_predicton_muegge_10, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (Muegge 10 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_muegge_test, test_predicton_muegge_20, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (Muegge 20 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_muegge_test, test_predicton_muegge_30, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (Muegge 30 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_bro_test, test_prediction_bro_10, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (bRo5 10 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_bro_test, test_prediction_bro_20, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (bRo5 20 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_bro_test, test_prediction_bro_30, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier (bRo5 30 Trees)")
plt.grid(True)
plt.show()

In [ ]:
classes = np.unique(target_lipinski_test)
n_classes = len(classes)

lipinski_test_bin = label_binarize(target_lipinski_test, classes=classes)
lipinski_10_score = model_lipinski_10.predict_proba(attributes_lipinski_test)

fpr_micro, tpr_micro, _ = roc_curve(lipinski_test_bin.ravel(), lipinski_10_score.ravel())
auc_micro = roc_auc_score(lipinski_test_bin, lipinski_10_score, average="micro")
auc_macro = roc_auc_score(lipinski_test_bin, lipinski_10_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Lipinski 10 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_lipinski_test)
n_classes_lipinski = len(classes)

lipinski_test_bin = label_binarize(target_lipinski_test, classes=classes)
lipinski_20_score = model_lipinski_20.predict_proba(attributes_lipinski_test)

fpr_micro, tpr_micro, _ = roc_curve(lipinski_test_bin.ravel(), lipinski_20_score.ravel())
auc_micro = roc_auc_score(lipinski_test_bin, lipinski_20_score, average="micro")
auc_macro = roc_auc_score(lipinski_test_bin, lipinski_20_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Lipinski 20 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_lipinski_test)
n_classes_lipinski = len(classes)

lipinski_test_bin = label_binarize(target_lipinski_test, classes=classes)
lipinski_30_score = model_lipinski_30.predict_proba(attributes_lipinski_test)

fpr_micro, tpr_micro, _ = roc_curve(lipinski_test_bin.ravel(), lipinski_30_score.ravel())
auc_micro = roc_auc_score(lipinski_test_bin, lipinski_30_score, average="micro")
auc_macro = roc_auc_score(lipinski_test_bin, lipinski_30_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Lipinski 30 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_muegge_test)
n_classes_muegge = len(classes)

muegge_test_bin = label_binarize(target_muegge_test, classes=classes)
muegge_10_score = model_muegge_10.predict_proba(attributes_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(muegge_test_bin.ravel(), muegge_10_score.ravel())
auc_micro = roc_auc_score(muegge_test_bin, muegge_10_score, average="micro")
auc_macro = roc_auc_score(muegge_test_bin, muegge_10_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Muegge 10 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_muegge_test)
n_classes_muegge = len(classes)

muegge_test_bin = label_binarize(target_muegge_test, classes=classes)
muegge_20_score = model_muegge_20.predict_proba(attributes_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(muegge_test_bin.ravel(), muegge_20_score.ravel())
auc_micro = roc_auc_score(muegge_test_bin, muegge_20_score, average="micro")
auc_macro = roc_auc_score(muegge_test_bin, muegge_20_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Muegge 20 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_muegge_test)
n_classes_muegge = len(classes)

muegge_test_bin = label_binarize(target_muegge_test, classes=classes)
muegge_30_score = model_muegge_30.predict_proba(attributes_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(muegge_test_bin.ravel(), muegge_30_score.ravel())
auc_micro = roc_auc_score(muegge_test_bin, muegge_30_score, average="micro")
auc_macro = roc_auc_score(muegge_test_bin, muegge_30_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Muegge 30 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_bro_test)
n_classes_bro = len(classes)

bro_test_bin = label_binarize(target_bro_test, classes=classes)
bro_10_score = model_bro_10.predict_proba(attributes_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(bro_test_bin.ravel(), bro_10_score.ravel())
auc_micro = roc_auc_score(bro_test_bin, bro_10_score, average="micro")
auc_macro = roc_auc_score(bro_test_bin, bro_10_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC bRo5 10 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_bro_test)
n_classes_bro = len(classes)

bro_test_bin = label_binarize(target_bro_test, classes=classes)
bro_20_score = model_bro_20.predict_proba(attributes_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(bro_test_bin.ravel(), bro_20_score.ravel())
auc_micro = roc_auc_score(bro_test_bin, bro_20_score, average="micro")
auc_macro = roc_auc_score(bro_test_bin, bro_20_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC bRo5 20 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_bro_test)
n_classes_bro = len(classes)

bro_test_bin = label_binarize(target_bro_test, classes=classes)
bro_30_score = model_bro_30.predict_proba(attributes_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(bro_test_bin.ravel(), bro_30_score.ravel())
auc_micro = roc_auc_score(bro_test_bin, bro_30_score, average="micro")
auc_macro = roc_auc_score(bro_test_bin, bro_30_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC bRo5 30 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
test_dataset = pd.DataFrame.from_dict(molecular_descriptors_test_peptides, orient='index').T

In [ ]:
test_dataset = test_dataset.rename(columns={'MolWt': 'mw', 'MolLogP': 'xlogp', 'NumHDonors': 'hbonddonor', 'NumHAcceptors': 'hbondacc', 'TPSA': 'polararea', 'NumRotatableBonds': 'rotbonds'})
test_dataset_for_work = test_dataset[['mw', 'xlogp', 'hbonddonor', 'hbondacc', 'polararea', 'rotbonds']]
test_dataset_for_work_lipinski = test_dataset[['mw', 'xlogp', 'hbonddonor', 'hbondacc']]

In [ ]:
peptide_names = ['ML1', 'ML2', 'ML3', 'ML4', 'ML5', 'ML6', 'ML7', 'ML8', 'ML9', 'ML10', 'ML11', 'ML12', 'ML13', 'ML14', 'ML15', 'ML16', 'ML17', 'ML18', 'ML19', 'ML20', 'ML21', 'ML22', 'ML23', 'ML24', 'ML25', 'ML26']

In [ ]:
tested_molecules = {'tested_molecules': peptide_names}

In [ ]:
prediction_lipinski_10 = model_lipinski_10.predict(test_dataset_for_work_lipinski)

In [ ]:
prediction_lipinski_20 = model_lipinski_20.predict(test_dataset_for_work_lipinski)

In [ ]:
prediction_lipinski_30 = model_lipinski_30.predict(test_dataset_for_work_lipinski)

In [ ]:
prediction_bro_10 = model_bro_10.predict(test_dataset_for_work)

In [ ]:
prediction_bro_20 = model_bro_20.predict(test_dataset_for_work)

In [ ]:
prediction_bro_30 = model_bro_30.predict(test_dataset_for_work)

In [ ]:
prediction_muegge_10 = model_muegge_10.predict(test_dataset_for_work)

In [ ]:
prediction_muegge_20 = model_muegge_20.predict(test_dataset_for_work)

In [ ]:
prediction_muegge_30 = model_muegge_30.predict(test_dataset_for_work)

In [ ]:
predicted_violations_peptides_lipinski_10 = []

for predicted_value in prediction_lipinski_10:
    predicted_violations_peptides_lipinski_10.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_lipinski_20 = []

for predicted_value in prediction_lipinski_20:
    predicted_violations_peptides_lipinski_20.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_lipinski_30 = []

for predicted_value in prediction_lipinski_30:
    predicted_violations_peptides_lipinski_30.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_bro_10 = []

for predicted_value in prediction_bro_10:
    predicted_violations_peptides_bro_10.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_bro_20 = []

for predicted_value in prediction_bro_20:
    predicted_violations_peptides_bro_20.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_bro_30 = []

for predicted_value in prediction_bro_30:
    predicted_violations_peptides_bro_30.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_muegge_10 = []

for predicted_value in prediction_muegge_10:
    predicted_violations_peptides_muegge_10.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_muegge_20 = []

for predicted_value in prediction_muegge_20:
    predicted_violations_peptides_muegge_20.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_muegge_30 = []

for predicted_value in prediction_muegge_30:
    predicted_violations_peptides_muegge_30.append(int(predicted_value))

In [ ]:
print(predicted_violations_peptides_lipinski_10)
print(predicted_violations_peptides_lipinski_20)
print(predicted_violations_peptides_lipinski_30)
print(predicted_violations_peptides_bro_10)
print(predicted_violations_peptides_bro_20)
print(predicted_violations_peptides_bro_30)
print(predicted_violations_peptides_muegge_10)
print(predicted_violations_peptides_muegge_20)
print(predicted_violations_peptides_muegge_30)

In [ ]:
lipinski_violations_dict_10 = {'nviolations_lipinski_rule_10': predicted_violations_peptides_lipinski_10}
lipinski_violations_dict_20 = {'nviolations_lipinski_rule_20': predicted_violations_peptides_lipinski_20}
lipinski_violations_dict_30 = {'nviolations_lipinski_rule_30': predicted_violations_peptides_lipinski_30}

In [ ]:
bro_violations_dict_10 = {'nviolations_bro_rule': predicted_violations_peptides_bro_10}
bro_violations_dict_20 = {'nviolations_bro_rule': predicted_violations_peptides_bro_20}
bro_violations_dict_30 = {'nviolations_bro_rule': predicted_violations_peptides_bro_30}

In [ ]:
muegge_violations_10_dict = {'nviolations_muegge_rule_10': predicted_violations_peptides_muegge_10}
muegge_violations_20_dict = {'nviolations_muegge_rule_20': predicted_violations_peptides_muegge_20}
muegge_violations_30_dict = {'nviolations_muegge_rule_30': predicted_violations_peptides_muegge_30}

In [ ]:
peptide_predictions_dataset_lipinski = pd.concat([pd.DataFrame(tested_molecules), test_dataset_for_work_lipinski, pd.DataFrame(lipinski_violations_dict_10), pd.DataFrame(lipinski_violations_dict_20), pd.DataFrame(lipinski_violations_dict_30)], axis=1)

In [ ]:
peptide_predictions_bro_dataset = pd.concat([pd.DataFrame(tested_molecules), test_dataset_for_work, pd.DataFrame(bro_violations_dict_10), pd.DataFrame(bro_violations_dict_20), pd.DataFrame(bro_violations_dict_30)], axis=1)

In [ ]:
peptide_predictions_muegge_dataset = pd.concat([pd.DataFrame(tested_molecules), test_dataset_for_work, pd.DataFrame(muegge_violations_10_dict), pd.DataFrame(muegge_violations_20_dict), pd.DataFrame(muegge_violations_30_dict)], axis=1)

In [ ]:
swissadme_peptides = pd.read_csv('datasets/swissadme_peptides.csv')

In [ ]:
comparison_table_peptides_lipinski = pd.concat([peptide_predictions_dataset_lipinski[['tested_molecules', 'nviolations_lipinski_rule_10', 'nviolations_lipinski_rule_20', 'nviolations_lipinski_rule_30']], swissadme_peptides[['Lipinski #violations']]], axis=1)

In [ ]:
comparison_table_peptides_muegge = pd.concat([peptide_predictions_muegge_dataset[['tested_molecules', 'nviolations_muegge_rule_10', 'nviolations_muegge_rule_20', 'nviolations_muegge_rule_30']], swissadme_peptides[['Muegge #violations']]], axis=1)

In [ ]:
dataset_smiles_violations = dataset_with_nviolations[['smiles', 'nviolations_lipinski_rule', 'nviolations_muegge_rule', 'nviolations_bro_rule']]

In [ ]:
dataset_smiles_violations

In [ ]:
list_of_smiles = [smile for smile in dataset_with_nviolations['smiles']]
list_lipinski_violations = [violation for violation in dataset_with_nviolations['nviolations_lipinski_rule']]
list_bro_violations = [violation for violation in dataset_with_nviolations['nviolations_bro_rule']]
list_muegge_violations = [violation for violation in dataset_with_nviolations['nviolations_muegge_rule']]

In [ ]:
fingerprint_generator = rdFingerprintGenerator.GetMorganGenerator(radius=4, fpSize=1024)

fingerprints_list = []
target_lipinski_values = []
target_bro_values = []
target_muegge_values = []

for _, row in dataset_smiles_violations.iterrows():

    smile = row['smiles']
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
        fp = fingerprint_generator.GetFingerprint(mol)
        fingerprints_list.append(np.array(fp))
        target_lipinski_values.append(row['nviolations_lipinski_rule'])
        target_bro_values.append(row['nviolations_bro_rule'])
        target_muegge_values.append(row['nviolations_muegge_rule'])

fingerprints_array = np.array(fingerprints_list, dtype=np.float32)

In [ ]:
violations_lipinski_vectors = {'nviolations_lipinski_rule': target_lipinski_values}
violations_bro_vectors = {'nviolations_bro_rule': target_bro_values}
violations_muegge_vectors = {'nviolations_muegge_rule': target_muegge_values}

In [ ]:
smiles_vectors = pd.DataFrame(fingerprints_array)

In [ ]:
smiles_vectors_violations = pd.concat([smiles_vectors, pd.DataFrame(violations_lipinski_vectors), pd.DataFrame(violations_bro_vectors), pd.DataFrame(violations_muegge_vectors)], axis=1)

In [ ]:
smiles_vectors_lipinski_train, smiles_vectors_lipinski_test, target_lipinski_vectors_train, target_lipinski_vectors_test = train_test_split(smiles_vectors_violations.iloc[:, 1:1025], smiles_vectors_violations['nviolations_lipinski_rule'], test_size=0.2, random_state=42)

In [ ]:
smiles_vectors_bro_train, smiles_vectors_bro_test, target_bro_vectors_train, target_bro_vectors_test = train_test_split(smiles_vectors_violations.iloc[:, 1:1025], smiles_vectors_violations['nviolations_bro_rule'], test_size=0.2, random_state=42)

In [ ]:
smiles_vectors_muegge_train, smiles_vectors_muegge_test, target_muegge_vectors_train, target_muegge_vectors_test = train_test_split(smiles_vectors_violations.iloc[:, 1:1025], smiles_vectors_violations['nviolations_muegge_rule'], test_size=0.2, random_state=42)

In [ ]:
smiles_vectors_lipinski_train.columns = smiles_vectors_lipinski_train.columns.astype(str)
smiles_vectors_bro_train.columns = smiles_vectors_bro_train.columns.astype(str)
smiles_vectors_muegge_train.columns = smiles_vectors_muegge_train.columns.astype(str)

In [ ]:
rf_lipinski_vectors = RandomForestClassifier()

smiles_vectors_lipinski_train.columns = smiles_vectors_lipinski_train.columns.astype(str)

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_lipinski_vectors, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_lipinski_train, target_lipinski_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

In [ ]:
rf_lipinski_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(precision_score, average='macro')

grid_search = GridSearchCV(estimator=rf_lipinski_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_lipinski_train, target_lipinski_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best precision:", grid_search.best_score_)

In [ ]:
rf_lipinski_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(recall_score, average='macro')

grid_search = GridSearchCV(estimator=rf_lipinski_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_lipinski_train, target_lipinski_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)

In [ ]:
rf_lipinski_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(estimator=rf_lipinski_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_lipinski_train, target_lipinski_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best F1-score:", grid_search.best_score_)

In [ ]:
rf_bro_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_bro_vectors, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_bro_train, target_bro_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

In [ ]:
rf_bro_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(precision_score, average='macro')

grid_search = GridSearchCV(estimator=rf_bro_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_bro_train, target_bro_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best precision:", grid_search.best_score_)

In [ ]:
rf_bro_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(recall_score, average='macro')

grid_search = GridSearchCV(estimator=rf_bro_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_bro_train, target_bro_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)

In [ ]:
rf_bro_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(estimator=rf_bro_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_bro_train, target_bro_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best F1-score:", grid_search.best_score_)

In [ ]:
rf_muegge_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_muegge_vectors, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_muegge_train, target_muegge_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

In [ ]:
rf_muegge_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(precision_score, average='macro')

grid_search = GridSearchCV(estimator=rf_muegge_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_muegge_train, target_muegge_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best precision:", grid_search.best_score_)

In [ ]:
rf_muegge_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(recall_score, average='macro')

grid_search = GridSearchCV(estimator=rf_muegge_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_muegge_train, target_muegge_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)

In [ ]:
rf_muegge_vectors = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True, False]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(estimator=rf_muegge_vectors, param_grid=param_grid,
                           cv=5, scoring=scorer, n_jobs=1, verbose=2, error_score='raise')

grid_search.fit(smiles_vectors_muegge_train, target_muegge_vectors_train)

print("Best parameters:", grid_search.best_params_)
print("Best F1-score:", grid_search.best_score_)

In [ ]:
model_lipinski_vectors_10 = RandomForestClassifier(n_estimators=10)

In [ ]:
model_lipinski_vectors_20 = RandomForestClassifier(n_estimators=20)

In [ ]:
model_lipinski_vectors_30 = RandomForestClassifier(n_estimators=30)

In [ ]:
model_bro_vectors_10 = RandomForestClassifier(n_estimators=10)

In [ ]:
model_bro_vectors_20 = RandomForestClassifier(n_estimators=20)

In [ ]:
model_bro_vectors_30 = RandomForestClassifier(n_estimators=30)

In [ ]:
model_muegge_vectors_10 = RandomForestClassifier(n_estimators=10)

In [ ]:
model_muegge_vectors_20 = RandomForestClassifier(n_estimators=20)

In [ ]:
model_muegge_vectors_30 = RandomForestClassifier(n_estimators=30)

In [ ]:
model_lipinski_vectors_10.fit(smiles_vectors_lipinski_train, target_lipinski_vectors_train)

In [ ]:
model_lipinski_vectors_20.fit(smiles_vectors_lipinski_train, target_lipinski_vectors_train)

In [ ]:
model_lipinski_vectors_30.fit(smiles_vectors_lipinski_train, target_lipinski_vectors_train)

In [ ]:
model_bro_vectors_10.fit(smiles_vectors_bro_train, target_bro_vectors_train)

In [ ]:
model_bro_vectors_20.fit(smiles_vectors_bro_train, target_bro_vectors_train)

In [ ]:
model_bro_vectors_30.fit(smiles_vectors_bro_train, target_bro_vectors_train)

In [ ]:
model_muegge_vectors_10.fit(smiles_vectors_muegge_train, target_muegge_vectors_train)

In [ ]:
model_muegge_vectors_20.fit(smiles_vectors_muegge_train, target_muegge_vectors_train)

In [ ]:
model_muegge_vectors_30.fit(smiles_vectors_muegge_train, target_muegge_vectors_train)

In [ ]:
smiles_vectors_lipinski_test.columns = smiles_vectors_lipinski_test.columns.astype(str)
smiles_vectors_bro_test.columns = smiles_vectors_bro_test.columns.astype(str)
smiles_vectors_muegge_test.columns = smiles_vectors_muegge_test.columns.astype(str)

In [ ]:
test_prediction_lipinski_vectors_10 = model_lipinski_vectors_10.predict(smiles_vectors_lipinski_test)

In [ ]:
test_prediction_lipinski_vectors_20 = model_lipinski_vectors_20.predict(smiles_vectors_lipinski_test)

In [ ]:
test_prediction_lipinski_vectors_30 = model_lipinski_vectors_30.predict(smiles_vectors_lipinski_test)

In [ ]:
test_prediction_bro_vectors_10 = model_bro_vectors_10.predict(smiles_vectors_bro_test)

In [ ]:
test_prediction_bro_vectors_20 = model_bro_vectors_20.predict(smiles_vectors_bro_test)

In [ ]:
test_prediction_bro_vectors_30 = model_bro_vectors_30.predict(smiles_vectors_bro_test)

In [ ]:
test_predicton_muegge_vectors_10 = model_muegge_vectors_10.predict(smiles_vectors_muegge_test)

In [ ]:
test_predicton_muegge_vectors_20 = model_muegge_vectors_20.predict(smiles_vectors_muegge_test)

In [ ]:
test_predicton_muegge_vectors_30 = model_muegge_vectors_30.predict(smiles_vectors_muegge_test)

In [ ]:
accuracy_lipinski_vectors_model_10 = accuracy_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_10)
accuracy_lipinski_vectors_model_20 = accuracy_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_20)
accuracy_lipinski_vectors_model_30 = accuracy_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_30)

In [ ]:
accuracy_bro_vectors_model_10 = accuracy_score(target_bro_vectors_test, test_prediction_bro_vectors_10)
accuracy_bro_vectors_model_20 = accuracy_score(target_bro_vectors_test, test_prediction_bro_vectors_20)
accuracy_bro_vectors_model_30 = accuracy_score(target_bro_vectors_test, test_prediction_bro_vectors_30)

In [ ]:
accuracy_muegge_vectors_model_10 = accuracy_score(target_muegge_vectors_test, test_predicton_muegge_vectors_10)
accuracy_muegge_vectors_model_20 = accuracy_score(target_muegge_vectors_test, test_predicton_muegge_vectors_20)
accuracy_muegge_vectors_model_30 = accuracy_score(target_muegge_vectors_test, test_predicton_muegge_vectors_30)

In [ ]:
print("Accuracy of Lipinski Vectors model with 10 trees:", accuracy_lipinski_vectors_model_10)
print("Accuracy of Lipinski Vectors model with 20 trees:", accuracy_lipinski_vectors_model_20)
print("Accuracy of Lipinski Vectors model with 30 trees:", accuracy_lipinski_vectors_model_30)
print("Accuracy of Bro Vectors model with 10 trees:", accuracy_bro_vectors_model_10)
print("Accuracy of Bro Vectors model with 20 trees:", accuracy_bro_vectors_model_20)
print("Accuracy of Bro Vectors model with 30 trees:", accuracy_bro_vectors_model_30)
print("Accuracy of Muegge Vectors model with 10 trees:", accuracy_muegge_vectors_model_10)
print("Accuracy of Muegge Vectors model with 20 trees:", accuracy_muegge_vectors_model_20)
print("Accuracy of Muegge Vectors model with 30 trees:", accuracy_muegge_vectors_model_30)

In [ ]:
precision_lipinski_vectors_model_10 = precision_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_10, average='weighted')
precision_lipinski_vectors_model_20 = precision_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_20, average='weighted')
precision_lipinski_vectors_model_30 = precision_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_30, average='weighted')

In [ ]:
precision_bro_vectors_model_10 = precision_score(target_bro_vectors_test, test_prediction_bro_vectors_10, average='weighted')
precision_bro_vectors_model_20 = precision_score(target_bro_vectors_test, test_prediction_bro_vectors_20, average='weighted')
precision_bro_vectors_model_30 = precision_score(target_bro_vectors_test, test_prediction_bro_vectors_30, average='weighted')

In [ ]:
precision_muegge_vectors_model_10 = precision_score(target_muegge_vectors_test, test_predicton_muegge_vectors_10, average='weighted')
precision_muegge_vectors_model_20 = precision_score(target_muegge_vectors_test, test_predicton_muegge_vectors_20, average='weighted')
precision_muegge_vectors_model_30 = precision_score(target_muegge_vectors_test, test_predicton_muegge_vectors_30, average='weighted')

In [ ]:
print("Precision of Lipinski Vectors model with 10 trees:", precision_lipinski_vectors_model_10)
print("Precision of Lipinski Vectors model with 20 trees:", precision_lipinski_vectors_model_20)
print("Precision of Lipinski Vectors model with 30 trees:", precision_lipinski_vectors_model_30)
print("Precision of Bro Vectors model with 10 trees:", precision_bro_vectors_model_10)
print("Precision of Bro Vectors model with 20 trees:", precision_bro_vectors_model_20)
print("Precision of Bro Vectors model with 30 trees:", precision_bro_vectors_model_30)
print("Precision of Muegge Vectors model with 10 trees:", precision_muegge_vectors_model_10)
print("Precision of Muegge Vectors model with 20 trees:", precision_muegge_vectors_model_20)
print("Precision of Muegge Vectors model with 30 trees:", precision_muegge_vectors_model_30)

In [ ]:
recall_lipinski_vectors_10 = recall_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_10, average='weighted')
recall_lipinski_vectors_20 = recall_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_20, average='weighted')
recall_lipinski_vectors_30 = recall_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_30, average='weighted')

In [ ]:
recall_bro_vectors_10 = recall_score(target_bro_vectors_test, test_prediction_bro_vectors_10, average='weighted')
recall_bro_vectors_20 = recall_score(target_bro_vectors_test, test_prediction_bro_vectors_20, average='weighted')
recall_bro_vectors_30 = recall_score(target_bro_vectors_test, test_prediction_bro_vectors_30, average='weighted')

In [ ]:
recall_muegge_vectors_10 = recall_score(target_muegge_vectors_test, test_predicton_muegge_vectors_10, average='weighted')
recall_muegge_vectors_20 = recall_score(target_muegge_vectors_test, test_predicton_muegge_vectors_20, average='weighted')
recall_muegge_vectors_30 = recall_score(target_muegge_vectors_test, test_predicton_muegge_vectors_30, average='weighted')

In [ ]:
print("Recall of Lipinski Vectors model with 10 trees:", recall_lipinski_vectors_10)
print("Recall of Lipinski Vectors model with 20 trees:", recall_lipinski_vectors_20)
print("Recall of Lipinski Vectors model with 30 trees:", recall_lipinski_vectors_30)
print("Recall of Bro Vectors model with 10 trees:", recall_bro_vectors_10)
print("Recall of Bro Vectors model with 20 trees:", recall_bro_vectors_20)
print("Recall of Bro Vectors model with 30 trees:", recall_bro_vectors_30)
print("Recall of Muegge Vectors model with 10 trees:", recall_muegge_vectors_10)
print("Recall of Muegge Vectors model with 20 trees:", recall_muegge_vectors_20)
print("Recall of Muegge Vectors model with 30 trees:", recall_muegge_vectors_30)

In [ ]:
f1_score_lipinski_vectors_10 = f1_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_10, average='weighted')
f1_score_lipinski_vectors_20 = f1_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_20, average='weighted')
f1_score_lipinski_vectors_30 = f1_score(target_lipinski_vectors_test, test_prediction_lipinski_vectors_30, average='weighted')

In [ ]:
f1_score_bro_vectors_10 = f1_score(target_bro_vectors_test, test_prediction_bro_vectors_10, average='weighted')
f1_score_bro_vectors_20 = f1_score(target_bro_vectors_test, test_prediction_bro_vectors_20, average='weighted')
f1_score_bro_vectors_30 = f1_score(target_bro_vectors_test, test_prediction_bro_vectors_30, average='weighted')

In [ ]:
f1_score_muegge_vectors_10 = f1_score(target_muegge_vectors_test, test_predicton_muegge_vectors_10, average='weighted')
f1_score_muegge_vectors_20 = f1_score(target_muegge_vectors_test, test_predicton_muegge_vectors_20, average='weighted')
f1_score_muegge_vectors_30 = f1_score(target_muegge_vectors_test, test_predicton_muegge_vectors_30, average='weighted')

In [ ]:
print("F1 Score of Lipinski Vectors model with 10 trees:", f1_score_lipinski_vectors_10)
print("F1 Score of Lipinski Vectors model with 20 trees:", f1_score_lipinski_vectors_20)
print("F1 Score of Lipinski Vectors model with 30 trees:", f1_score_lipinski_vectors_30)
print("F1 Score of Bro Vectors model with 10 trees:", f1_score_bro_vectors_10)
print("F1 Score of Bro Vectors model with 20 trees:", f1_score_bro_vectors_20)
print("F1 Score of Bro Vectors model with 30 trees:", f1_score_bro_vectors_30)
print("F1 Score of Muegge Vectors model with 10 trees:", f1_score_muegge_vectors_10)
print("F1 Score of Muegge Vectors model with 20 trees:", f1_score_muegge_vectors_20)
print("F1 Score of Muegge Vectors model with 30 trees:", f1_score_muegge_vectors_30)

In [ ]:
plt.scatter(target_lipinski_vectors_test, test_prediction_lipinski_vectors_10, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (Lipinski 10 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_lipinski_vectors_test, test_prediction_lipinski_vectors_20, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (Lipinski 20 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_lipinski_vectors_test, test_prediction_lipinski_vectors_30, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (Lipinski 30 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_muegge_vectors_test, test_predicton_muegge_vectors_10, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (Muegge 10 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_muegge_vectors_test, test_predicton_muegge_vectors_20, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (Muegge 20 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_muegge_vectors_test, test_predicton_muegge_vectors_30, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (Muegge 30 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_bro_vectors_test, test_prediction_bro_vectors_10, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (bRo5 10 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_bro_vectors_test, test_prediction_bro_vectors_20, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (bRo5 20 Trees)")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(target_bro_vectors_test, test_prediction_bro_vectors_30, alpha=0.5, color='black')
plt.xlabel("Test Values")
plt.ylabel("Predicted Values")
plt.title("Scatter plot: Random Forest Classifier Vectors (bRo5 30 Trees)")
plt.grid(True)
plt.show()

In [ ]:
classes = np.unique(target_lipinski_vectors_test)
n_classes = len(classes)

lipinski_test_bin = label_binarize(target_lipinski_vectors_test, classes=classes)
lipinski_10_score = model_lipinski_vectors_10.predict_proba(smiles_vectors_lipinski_test)

fpr_micro, tpr_micro, _ = roc_curve(lipinski_test_bin.ravel(), lipinski_10_score.ravel())
auc_micro = roc_auc_score(lipinski_test_bin, lipinski_10_score, average="micro")
auc_macro = roc_auc_score(lipinski_test_bin, lipinski_10_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Lipinski Vectors 10 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_lipinski_vectors_test)
n_classes_lipinski = len(classes)

lipinski_test_bin = label_binarize(target_lipinski_vectors_test, classes=classes)
lipinski_20_score = model_lipinski_vectors_20.predict_proba(smiles_vectors_lipinski_test)

fpr_micro, tpr_micro, _ = roc_curve(lipinski_test_bin.ravel(), lipinski_20_score.ravel())
auc_micro = roc_auc_score(lipinski_test_bin, lipinski_20_score, average="micro")
auc_macro = roc_auc_score(lipinski_test_bin, lipinski_20_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Lipinski Vectors 20 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_lipinski_vectors_test)
n_classes_lipinski = len(classes)

lipinski_test_bin = label_binarize(target_lipinski_vectors_test, classes=classes)
lipinski_30_score = model_lipinski_vectors_30.predict_proba(smiles_vectors_lipinski_test)

fpr_micro, tpr_micro, _ = roc_curve(lipinski_test_bin.ravel(), lipinski_30_score.ravel())
auc_micro = roc_auc_score(lipinski_test_bin, lipinski_30_score, average="micro")
auc_macro = roc_auc_score(lipinski_test_bin, lipinski_30_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Lipinski Vectors 30 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_muegge_vectors_test)
n_classes_muegge = len(classes)

muegge_test_bin = label_binarize(target_muegge_vectors_test, classes=classes)
muegge_10_score = model_muegge_vectors_10.predict_proba(smiles_vectors_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(muegge_test_bin.ravel(), muegge_10_score.ravel())
auc_micro = roc_auc_score(muegge_test_bin, muegge_10_score, average="micro")
auc_macro = roc_auc_score(muegge_test_bin, muegge_10_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Muegge Vectors 10 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_muegge_vectors_test)
n_classes_muegge = len(classes)

muegge_test_bin = label_binarize(target_muegge_vectors_test, classes=classes)
muegge_20_score = model_muegge_vectors_20.predict_proba(smiles_vectors_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(muegge_test_bin.ravel(), muegge_20_score.ravel())
auc_micro = roc_auc_score(muegge_test_bin, muegge_20_score, average="micro")
auc_macro = roc_auc_score(muegge_test_bin, muegge_20_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Muegge Vectors 20 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_muegge_vectors_test)
n_classes_muegge = len(classes)

muegge_test_bin = label_binarize(target_muegge_vectors_test, classes=classes)
muegge_30_score = model_muegge_vectors_30.predict_proba(smiles_vectors_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(muegge_test_bin.ravel(), muegge_30_score.ravel())
auc_micro = roc_auc_score(muegge_test_bin, muegge_30_score, average="micro")
auc_macro = roc_auc_score(muegge_test_bin, muegge_30_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Muegge Vectors 30 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_bro_vectors_test)
n_classes_bro = len(classes)

bro_test_bin = label_binarize(target_bro_vectors_test, classes=classes)
bro_10_score = model_bro_vectors_10.predict_proba(smiles_vectors_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(bro_test_bin.ravel(), bro_10_score.ravel())
auc_micro = roc_auc_score(bro_test_bin, bro_10_score, average="micro")
auc_macro = roc_auc_score(bro_test_bin, bro_10_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC bRo5 Vectors 10 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_bro_vectors_test)
n_classes_bro = len(classes)

bro_test_bin = label_binarize(target_bro_vectors_test, classes=classes)
bro_20_score = model_bro_vectors_20.predict_proba(smiles_vectors_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(bro_test_bin.ravel(), bro_20_score.ravel())
auc_micro = roc_auc_score(bro_test_bin, bro_20_score, average="micro")
auc_macro = roc_auc_score(bro_test_bin, bro_20_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC bRo5 Vectors 20 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
classes = np.unique(target_bro_vectors_test)
n_classes_bro = len(classes)

bro_test_bin = label_binarize(target_bro_vectors_test, classes=classes)
bro_30_score = model_bro_vectors_30.predict_proba(smiles_vectors_muegge_test)

fpr_micro, tpr_micro, _ = roc_curve(bro_test_bin.ravel(), bro_30_score.ravel())
auc_micro = roc_auc_score(bro_test_bin, bro_30_score, average="micro")
auc_macro = roc_auc_score(bro_test_bin, bro_30_score, average="macro")

plt.plot(fpr_micro, tpr_micro, label=f"Micro-average ROC (AUC = {auc_micro:.2f})", color='black')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC bRo5 Vectors 30 Trees")
plt.legend()
plt.grid(True)
plt.show()

print(f"Macro-average AUC: {auc_macro:.3f}")
print(f"Micro-average AUC: {auc_micro:.3f}")

In [ ]:
peptide_molecules_smiles = ['N[C@@H](CCCNC(=N)N)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)O', 'NC(CCNC(=N)N)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)O', 'NC(CNC(=N)N)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)O', 'N[C@@H](CCONC(=N)N)C(=O)OCC(=O)N[C@@H](CC(=O)O)C(=O)O', 'NCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)[C@@H](N)CCCNC(=N)N', 'NCCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)[C@@H](N)CCCNC(=N)N', 'NCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CCNC(=N)N', 'NCCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CCNC(=N)N', 'NCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CNC(=N)N', 'NCCCNC(=O)[C@H](CC(=O)O)NC(=O)CNC(=O)C(N)CNC(=N)N', 'NCCNC(=O)[C@H](CC(=O)O)NC(=O)COC(=O)[C@@H](N)CCONC(=N)N', 'NCCCNC(=O)[C@H](CC(=O)O)NC(=O)COC(=O)[C@@H](N)CCONC(=N)N', 'CC[C@H](C)[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(=O)O', 'CC[C@H](C)[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@H](Cc2ccc(F)cc2)NC(=O)[C@H](C)N)C(=O)O', 'C[C@H](N)C(=O)N[C@@H](Cc1ccc(F)cc1)C(=O)N2CCC[C@H]2C(=O)N[C@H](Cc3ccc(F)cc3)C(=O)O', 'CC(C)[C@H](NC(=O)[C@H](C)N)C(=O)N1CCC[C@H]1C(=O)N[C@H](Cc2ccc(F)cc2)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](N)CCCNC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](N)CCCCN)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCCN)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](N)CCCNC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CC[C@@H](C)[C@H](NC(=O)[C@H](Cc1ccc(F)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](N)CCONC(=N)N)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CCC(C)[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCONC(=N)N)NC(=O)[C@H](CCONC(=N)N)NC(=O)[C@@H](NC(=O)[C@@H]3CCCN3C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(C)CC)C(=O)N[C@@H](CC(C)C)C(=O)O', 'CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)C2CCCN2C(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](NC(=O)[C@@H]3CCCN3C(=O)[C@@H](NC(=O)[C@H](C)N)C(C)C)C(C)CC)C(=O)N[C@@H](CC(C)C)C(=O)O']

In [ ]:
peptides_molecules_smiles_dict = {'smiles': peptide_molecules_smiles}

In [ ]:
test_dataset_smiles = pd.concat([pd.DataFrame(peptides_molecules_smiles_dict), test_dataset_for_work], axis=1)

In [ ]:
fingerprint_generator_test = rdFingerprintGenerator.GetMorganGenerator(radius=4, fpSize=1024)

test_molecules_fingerprints_list = []

for _, row in test_dataset_smiles.iterrows():

    smile = row['smiles']
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
        fp = fingerprint_generator_test.GetFingerprint(mol)
        test_molecules_fingerprints_list.append(np.array(fp))

test_molecules_fingerprints_array = np.array(test_molecules_fingerprints_list, dtype=np.float32)

In [ ]:
test_molecules_fingerprints_df = pd.DataFrame(test_molecules_fingerprints_array)

In [ ]:
prediction_lipinski_vectors_10 = model_lipinski_vectors_10.predict(test_molecules_fingerprints_df)
prediction_lipinski_vectors_20 = model_lipinski_vectors_20.predict(test_molecules_fingerprints_df)
prediction_lipinski_vectors_30 = model_lipinski_vectors_30.predict(test_molecules_fingerprints_df)

In [ ]:
prediction_bro_vectors_10 = model_bro_vectors_10.predict(test_molecules_fingerprints_array)
prediction_bro_vectors_20 = model_bro_vectors_20.predict(test_molecules_fingerprints_array)
prediction_bro_vectors_30 = model_bro_vectors_30.predict(test_molecules_fingerprints_array)

In [ ]:
prediction_muegge_vectors_10 = model_muegge_vectors_10.predict(test_molecules_fingerprints_array)
prediction_muegge_vectors_20 = model_muegge_vectors_20.predict(test_molecules_fingerprints_array)
prediction_muegge_vectors_30 = model_muegge_vectors_30.predict(test_molecules_fingerprints_array)

In [ ]:
predicted_violations_peptides_lipinski_vectors_10 = []

for predicted_value in prediction_lipinski_vectors_10:
    predicted_violations_peptides_lipinski_vectors_10.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_lipinski_vectors_20 = []

for predicted_value in prediction_lipinski_vectors_20:
    predicted_violations_peptides_lipinski_vectors_20.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_lipinski_vectors_30 = []

for predicted_value in prediction_lipinski_vectors_30:
    predicted_violations_peptides_lipinski_vectors_30.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_bro_vectors_10 = []

for predicted_value in prediction_bro_vectors_10:
    predicted_violations_peptides_bro_vectors_10.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_bro_vectors_20 = []

for predicted_value in prediction_bro_vectors_20:
    predicted_violations_peptides_bro_vectors_20.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_bro_vectors_30 = []

for predicted_value in prediction_bro_vectors_30:
    predicted_violations_peptides_bro_vectors_30.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_muegge_vectors_10 = []

for predicted_value in prediction_muegge_vectors_10:
    predicted_violations_peptides_muegge_vectors_10.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_muegge_vectors_20 = []

for predicted_value in prediction_muegge_vectors_20:
    predicted_violations_peptides_muegge_vectors_20.append(int(predicted_value))

In [ ]:
predicted_violations_peptides_muegge_vectors_30 = []

for predicted_value in prediction_muegge_vectors_30:
    predicted_violations_peptides_muegge_vectors_30.append(int(predicted_value))

In [ ]:
lipinski_violations_vectors_dict_10 = {'nviolations_lipinski_rule_10': predicted_violations_peptides_lipinski_vectors_10}
lipinski_violations_vectors_dict_20 = {'nviolations_lipinski_rule_20': predicted_violations_peptides_lipinski_vectors_20}
lipinski_violations_vectors_dict_30 = {'nviolations_lipinski_rule_30': predicted_violations_peptides_lipinski_vectors_30}
bro_violations_dict_vectors_10 = {'nviolations_bro_rule': predicted_violations_peptides_bro_vectors_10}
bro_violations_dict_vectors_20 = {'nviolations_bro_rule': predicted_violations_peptides_bro_vectors_20}
bro_violations_dict_vectors_30 = {'nviolations_bro_rule': predicted_violations_peptides_bro_vectors_30}
muegge_violations_10_vectors_dict = {'nviolations_muegge_rule_10': predicted_violations_peptides_muegge_vectors_10}
muegge_violations_20_vectors_dict = {'nviolations_muegge_rule_20': predicted_violations_peptides_muegge_vectors_20}
muegge_violations_30_vectors_dict = {'nviolations_muegge_rule_30': predicted_violations_peptides_muegge_vectors_30}

In [ ]:
peptide_predictions_dataset_lipinski_vectors = pd.concat(
    [pd.DataFrame(lipinski_violations_vectors_dict_10),
     pd.DataFrame(lipinski_violations_vectors_dict_20), pd.DataFrame(lipinski_violations_vectors_dict_30)], axis=1)
peptide_predictions_bro_dataset_vectors = pd.concat(
    [pd.DataFrame(bro_violations_dict_vectors_10),
     pd.DataFrame(bro_violations_dict_vectors_20), pd.DataFrame(bro_violations_dict_vectors_30)], axis=1)
peptide_predictions_muegge_dataset_vectors = pd.concat(
    [pd.DataFrame(muegge_violations_10_vectors_dict),
     pd.DataFrame(muegge_violations_20_vectors_dict), pd.DataFrame(muegge_violations_30_vectors_dict)], axis=1)

In [ ]:
peptide_predictions_dataset_lipinski_vectors

In [ ]:
peptide_predictions_bro_dataset_vectors

In [ ]:
peptide_predictions_muegge_dataset_vectors